In [1]:
# https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git

In [2]:
#git remote add origin https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git
#git branch -M main
#git push -u origin main

In [3]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keras 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install Matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bokeh

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install skforecast --user

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install Tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install textblob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Data manipulation
# ==============================================================================
import numpy as np
from numpy import loadtxt
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk import download

from textblob import TextBlob


# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
#from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load


[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#from keras.models import Sequential

In [13]:
#from keras.layers import Dense 

In [14]:

# Spark
# import SparkSession library 
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import udf, col, lower, concat_ws, regexp_replace
from pyspark.sql.functions import split, size, length, broadcast, sum
from pyspark.sql.types import DoubleType, StructType, StructField, ArrayType
from pyspark.sql.types import StringType, IntegerType, TimestampType
from pyspark.sql.types import *
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.linalg import Vectors
import pyspark.sql.types as typ
import pyspark.sql.functions as fn
import pyspark.sql.functions as F


# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

In [15]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [16]:
# SparkContext 
sc.master

'local[*]'

In [17]:
# Creating a SparkSession  **from tutorial 4 - NN using Pyspark

In [18]:
spark = SparkSession.builder.appName("MySparkApplication").getOrCreate()

In [19]:
schema = StructType([
    StructField("Ids", IntegerType(), True),
    StructField("Date", StringType(), True),
    StructField("Flag", StringType(), True),
    StructField("User", StringType(), True),
    StructField("Text", StringType(), True)
])

In [20]:
# ****************************************************************************************************************
# *                                  Start with the Data                                                         *
# ****************************************************************************************************************

In [21]:
# Big headache and colapse, to read the dataset
# I was load the file from my hadoop (local)
#spark_df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [22]:
#Move the dataset fiel to HDFS path from my terminal:
#First check:  $fs -ls/user1  from /home/hduser/Documents/CA2/
#Next move :   $hadoop fs-put./ProjectTweets.scv/user1
#Check again : $fs -ls/user1
#I will use direct from the path, to have one of five V' (Velocity) 


#folowiong you can see the two way to load the dataset, 
#    (command)+(file://)+(/path/)+(filename)
#I will use direct from the path, to have one of five V' (Velocity) 

#Tweets_path = "/user1/ProjectTweets.csv"
#df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [23]:
spark_df = spark.read.csv("/user1/ProjectTweets.csv", header=False, inferSchema=True)

In [24]:
column_names = ['_c0', 'Ids', 'Date', 'Flag', 'User', 'Text']

In [25]:
for i, column_name in enumerate(column_names):
    spark_df = spark_df.withColumnRenamed("_c" + str(i),column_name)

In [26]:
#Showing the data
spark_df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       Ids|                Date|    Flag|           User|                Text|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [27]:
# display the total number of rows data
total_rows = spark_df.count()


In [28]:
formatted_total_rows = "{:,}".format(total_rows)
print("Total Rows:", formatted_total_rows)

Total Rows: 1,600,000


In [29]:
# Drop unnecessary column  ************************************************************

In [30]:
column_to_drop = ['Flag', 'Unnamed: 0']


In [31]:
spark_df = spark_df.drop('Flag')

In [32]:
spark_df.show()

+---+----------+--------------------+---------------+--------------------+
|_c0|       Ids|                Date|           User|                Text|
+---+----------+--------------------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K nope t...|
|  9|1467812025|Mon Apr 06 22:20:...|        mimismo|@twittera que me ...|
| 10|1467812416|Mon Apr 0

In [33]:
# Creating a UDF to apply VADER sentiment analysis to a Text column

In [34]:
schema = (
    StructType()
    .add("_c0","integer")
    .add("Ids","integer")
    .add("Date","string")
    .add("Flag", "string")
    .add("User","string")
    .add("Text", "string")
)

In [35]:
def analyze_sentiment(Text):
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(Text)
    return sentiment['compound']

In [36]:
#register the UDF

In [37]:
#sentiment_udf = udf(lambda text:spark_df("Text"), StringType()) 
sentiment_udf = udf(analyze_sentiment, StringType())

In [38]:
#Apply sentiment

In [39]:
spark_df = spark_df.withColumn("analyze_sentiment", sentiment_udf(spark_df["Text"]))

In [40]:
#print(analyze_sentiment.show())

In [90]:
spark_df.select("Text", "analyze_sentiment").show()

+--------------------+-----------------+
|                Text|analyze_sentiment|
+--------------------+-----------------+
|switchfoot httptw...|          -0.0173|
|is upset that he ...|            -0.75|
|kenichan i dived ...|           0.4939|
|my whole body fee...|            -0.25|
|nationwideclass n...|          -0.6597|
|kwesidei not the ...|              0.0|
|         need a hug |           0.4767|
|loltrish hey  lon...|            0.745|
|tatiana_k nope th...|              0.0|
|twittera que me m...|              0.0|
|spring break in p...|              0.0|
|i just repierced ...|              0.0|
|caregiving i coul...|          -0.5994|
|octolinz16 it it ...|          -0.1027|
|smarrison i would...|           0.3724|
|iamjazzyfizzle i ...|           0.4545|
|hollis death scen...|          -0.9081|
|about to file taxes |              0.0|
|lettya ahh ive al...|           0.6988|
|fakerpattypattz o...|           0.1779|
+--------------------+-----------------+
only showing top

In [42]:
#why i have Flag  again??
print(column_names[:6])

['_c0', 'Ids', 'Date', 'Flag', 'User', 'Text']


In [43]:
#to show

spark_df.drop('Flag').show()

+---+----------+--------------------+---------------+--------------------+-----------------+
|_c0|       Ids|                Date|           User|                Text|analyze_sentiment|
+---+----------+--------------------+---------------+--------------------+-----------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|          -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|            -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|           0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|            -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|          -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|              0.0|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |           0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo

In [44]:
print(spark_df.show())

+---+----------+--------------------+---------------+--------------------+-----------------+
|_c0|       Ids|                Date|           User|                Text|analyze_sentiment|
+---+----------+--------------------+---------------+--------------------+-----------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|          -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|            -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|           0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|            -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|          -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|              0.0|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |           0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo

In [45]:
#        Basic Feature Extraction

In [46]:
spark_df = spark_df.withColumn("word_count", size(split(spark_df["Text"], " ")))

In [47]:
# Number of Words  

In [48]:
spark_df.select("Text", "word_count").show()

+--------------------+----------+
|                Text|word_count|
+--------------------+----------+
|@switchfoot http:...|        20|
|is upset that he ...|        22|
|@Kenichan I dived...|        19|
|my whole body fee...|        11|
|@nationwideclass ...|        22|
|@Kwesidei not the...|         6|
|         Need a hug |         4|
|@LOLTrish hey  lo...|        24|
|@Tatiana_K nope t...|         7|
|@twittera que me ...|         6|
|spring break in p...|         8|
|I just re-pierced...|         6|
|@caregiving I cou...|        21|
|@octolinz16 It it...|        16|
|@smarrison i woul...|        23|
|@iamjazzyfizzle I...|        20|
|Hollis' death sce...|        19|
|about to file taxes |         5|
|@LettyA ahh ive a...|        12|
|@FakerPattyPattz ...|        13|
+--------------------+----------+
only showing top 20 rows



In [49]:
#Tutorial did't work because is a Panda code: TypeError: 'Column' object is not callable

#spark_df ['word_count'] = spark_df ['Text'].apply(lambda x: len(str(x).split("")))


In [50]:
#spark_df[['Text','word_count']].head()

In [51]:
# Number of Characters

In [52]:
#Tutorial did't work because is a Panda code:TypeError: 'Column' object is not callable
#Tweets['char_count'] = Tweets['Text'].str.len()
## this also includes spaces
#Tweets[['Text','char_count']].head()
#Tweets = Tweets.withColumn("char_count", length(Tweets["Text"]))
#Tweets.select("Text", "char_count").show()

In [53]:
spark_df = spark_df.withColumn("char_count", length(spark_df["Text"]))

In [54]:
spark_df.select("Text", "char_count").show()

+--------------------+----------+
|                Text|char_count|
+--------------------+----------+
|@switchfoot http:...|       115|
|is upset that he ...|       111|
|@Kenichan I dived...|        89|
|my whole body fee...|        47|
|@nationwideclass ...|       111|
|@Kwesidei not the...|        29|
|         Need a hug |        11|
|@LOLTrish hey  lo...|        99|
|@Tatiana_K nope t...|        36|
|@twittera que me ...|        25|
|spring break in p...|        43|
|I just re-pierced...|        26|
|@caregiving I cou...|        94|
|@octolinz16 It it...|        77|
|@smarrison i woul...|       117|
|@iamjazzyfizzle I...|       103|
|Hollis' death sce...|        93|
|about to file taxes |        20|
|@LettyA ahh ive a...|        64|
|@FakerPattyPattz ...|        79|
+--------------------+----------+
only showing top 20 rows



In [55]:
#Basic Pre-processing

In [56]:
spark_df = spark_df.withColumn("Text", concat_ws("", lower(col("Text"))))

In [57]:
spark_df.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|@switchfoot http:...|
|is upset that he ...|
|@kenichan i dived...|
|my whole body fee...|
|@nationwideclass ...|
|@kwesidei not the...|
|         need a hug |
|@loltrish hey  lo...|
|@tatiana_k nope t...|
|@twittera que me ...|
|spring break in p...|
|i just re-pierced...|
|@caregiving i cou...|
|@octolinz16 it it...|
|@smarrison i woul...|
|@iamjazzyfizzle i...|
|hollis' death sce...|
|about to file taxes |
|@lettya ahh ive a...|
|@fakerpattypattz ...|
+--------------------+
only showing top 20 rows



In [58]:
#Number  of Special characters 

In [59]:
#spark = SparkSession.builder.appName("SpecialCharactersCount").getOrCreate()

In [60]:
Special_Characters = ["@", "#"]

In [61]:
def count_Special_Characters(text):
    count = 0
    for char in Special_Characters:
        count += text.count(char)
    return count

In [62]:
count_Special_Characters_udf = udf(count_Special_Characters, IntegerType())

In [63]:
spark_df = spark_df.withColumn("Special_Characters_count", count_Special_Characters_udf(col("Text")))

In [64]:
selected_columns = spark_df.select("Text", "Special_Characters_count")

In [65]:
selected_columns.show()

+--------------------+------------------------+
|                Text|Special_Characters_count|
+--------------------+------------------------+
|@switchfoot http:...|                       1|
|is upset that he ...|                       0|
|@kenichan i dived...|                       1|
|my whole body fee...|                       0|
|@nationwideclass ...|                       1|
|@kwesidei not the...|                       1|
|         need a hug |                       0|
|@loltrish hey  lo...|                       1|
|@tatiana_k nope t...|                       1|
|@twittera que me ...|                       1|
|spring break in p...|                       0|
|i just re-pierced...|                       0|
|@caregiving i cou...|                       1|
|@octolinz16 it it...|                       1|
|@smarrison i woul...|                       1|
|@iamjazzyfizzle i...|                       2|
|hollis' death sce...|                       0|
|about to file taxes |                  

In [66]:
#Removing Punctuation 

In [67]:
spark_df = spark_df.withColumn("Text", regexp_replace(col("Text"), r'[^\w\s]', ''))

In [68]:
spark_df.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|is upset that he ...|
|kenichan i dived ...|
|my whole body fee...|
|nationwideclass n...|
|kwesidei not the ...|
|         need a hug |
|loltrish hey  lon...|
|tatiana_k nope th...|
|twittera que me m...|
|spring break in p...|
|i just repierced ...|
|caregiving i coul...|
|octolinz16 it it ...|
|smarrison i would...|
|iamjazzyfizzle i ...|
|hollis death scen...|
|about to file taxes |
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [69]:
#Count stop Word

In [70]:
stopwords = ["is", "the", "I"]

In [71]:
def count_stopwords(Text):
    words = Text.split()
#   return len([word for word in words if word in stopwords])

In [72]:
count_stopwords_udf = udf(count_stopwords, IntegerType())

In [73]:
spark_df = spark_df.withColumn("Stopwords", count_stopwords_udf(col("Text")))

In [74]:
#spark_df.select("Text", "Sentiment", "Stopwords").show()
#spark_df.show(8)
spark_df.select("Text", "Stopwords").show()

+--------------------+---------+
|                Text|Stopwords|
+--------------------+---------+
|switchfoot httptw...|     null|
|is upset that he ...|     null|
|kenichan i dived ...|     null|
|my whole body fee...|     null|
|nationwideclass n...|     null|
|kwesidei not the ...|     null|
|         need a hug |     null|
|loltrish hey  lon...|     null|
|tatiana_k nope th...|     null|
|twittera que me m...|     null|
|spring break in p...|     null|
|i just repierced ...|     null|
|caregiving i coul...|     null|
|octolinz16 it it ...|     null|
|smarrison i would...|     null|
|iamjazzyfizzle i ...|     null|
|hollis death scen...|     null|
|about to file taxes |     null|
|lettya ahh ive al...|     null|
|fakerpattypattz o...|     null|
+--------------------+---------+
only showing top 20 rows



In [75]:
#*****************************************************************************************************************
#*                                       T O K E N I Z A T I O N                                                 *
#*****************************************************************************************************************

In [76]:
#spark = SparkSession.builder.appName("Tokenization").getOrCreate()

In [77]:
#spark_df = spark.read.csv("/user1/ProjectTweets.csv", header=False, inferSchema=True)

In [78]:
#split into input (x) and output (y) variables
#x = spark_df[:.0:5]
#y = spark_df[:,5]


In [79]:
# Define the Keras model
#model = Sequential()

In [80]:
# function to Tokenization using TextBlob (2.7 Tokenization)
#def tokenize_spark_df(Text):
#    blob = TextBlob(Text)
#    return list(blob.words)


In [81]:
#Register
#tokenize_udf = udf(tokenize_spark_df)

In [82]:
#Apply teh UDF
#spark_df = spark_df.withColumn('Words', tokenize_udf(spark_df['_c5']))
#spark_df = spark_df.withColumn('tokenized_Tweets', tokenize_udf(spark_df['_c5']))

In [83]:
#spark_df.select('Words').show()
#spark_df[['Text', 'tokenized_Tweets']].show()

In [84]:
################################nothing run###########################

In [85]:
#following funtion to get sentiment polarity using TextBlob
def get_sentiment(Text):
    blob = TextBlob(Text)
    return blob.sentiment.polarity

In [86]:
#Register the UDF
sentiment_udf = udf(get_sentiment, FloatType())

In [87]:
#spark_df["sentiment"]=spark_df"["Text"].apply(lambda x: Textblob(x).sentimet[0])
spark_df = spark_df.withColumn('sentiment', sentiment_udf(spark_df['Text']))

In [89]:
spark_df.select("Text", "sentiment").show()

+--------------------+-----------+
|                Text|  sentiment|
+--------------------+-----------+
|switchfoot httptw...|        0.2|
|is upset that he ...|        0.0|
|kenichan i dived ...|        0.5|
|my whole body fee...|        0.2|
|nationwideclass n...|     -0.625|
|kwesidei not the ...|        0.2|
|         need a hug |        0.0|
|loltrish hey  lon...| 0.27333334|
|tatiana_k nope th...|        0.0|
|twittera que me m...|        0.0|
|spring break in p...|-0.21428572|
|i just repierced ...|        0.0|
|caregiving i coul...|        0.0|
|octolinz16 it it ...|        0.0|
|smarrison i would...|      0.075|
|iamjazzyfizzle i ...|        0.0|
|hollis death scen...|        0.0|
|about to file taxes |        0.0|
|lettya ahh ive al...|        0.5|
|fakerpattypattz o...|        0.0|
+--------------------+-----------+
only showing top 20 rows



In [92]:
# Function to categorize sentiment
def categorize_sentiment(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [93]:
#Apply function UDF
categorize_udf = udf(categorize_sentiment, StringType())

In [95]:
#check the score
spark_df = spark_df.withColumn('sentiment_category', categorize_udf(spark_df['sentiment']))

In [96]:
#show the categorized sentiment
spark_df.select('Text', 'sentiment_category').show()

+--------------------+------------------+
|                Text|sentiment_category|
+--------------------+------------------+
|switchfoot httptw...|          Positive|
|is upset that he ...|           Neutral|
|kenichan i dived ...|          Positive|
|my whole body fee...|          Positive|
|nationwideclass n...|          Negative|
|kwesidei not the ...|          Positive|
|         need a hug |           Neutral|
|loltrish hey  lon...|          Positive|
|tatiana_k nope th...|           Neutral|
|twittera que me m...|           Neutral|
|spring break in p...|          Negative|
|i just repierced ...|           Neutral|
|caregiving i coul...|           Neutral|
|octolinz16 it it ...|           Neutral|
|smarrison i would...|          Positive|
|iamjazzyfizzle i ...|           Neutral|
|hollis death scen...|           Neutral|
|about to file taxes |           Neutral|
|lettya ahh ive al...|          Positive|
|fakerpattypattz o...|           Neutral|
+--------------------+------------

In [ ]:
spark_df.show()

In [ ]:
# Averages Word ** Spark session **

In [ ]:
#######################################################

In [ ]:
#spark = SparkSession.builder.appName("AverageWordLength").getOrCreate()

In [ ]:
# define 

In [ ]:
#def avg_word(sentence):
#    words = sentence.split()
#    return sum(len(word) for word in words) / len(words)

In [ ]:
#avg_word_udf = udf(avg_word, DoubleType())

In [ ]:
#spark_df.withColumn("average_word_length", avg_word_udf(col("Sentiment")))

In [ ]:
#selected_columns = spark_df.select("Sentiment", "average_word_length")

In [ ]:
#selected_columns.show()

In [ ]:
#Tweets.show() 

In [ ]:
#**************************************************************

In [ ]:
#spark_df = spark.createDataFrame(Tweets)

In [ ]:
#def avg_word(sentence):
#    words = sentence.split()
#    return sum(len(word) for word in words) / len(words)

In [ ]:
#avg_word_udf = udf(avg_word, DoubleType())

In [ ]:
#Tweets_df  = Tweets_df .withColumn("average_word_length", avg_word_udf(col("_6")))

In [ ]:
#Tweets_df.show()

In [ ]:
#Schema for the  DataFrame
#schema = StructType([StructField("sentence", StringType(), True)])

In [ ]:
#  Crate a DataFrame

In [ ]:
#Tweets = [("Text",)]
#df = spark.createDataFrame(Tweets, schema)

In [ ]:
#Split the sentence

In [ ]:
#df =df.withColumn("words", split(col("sentence"), " "))
#df =df.withColumn("words_lengths", size(col("words")))

In [ ]:
# Calcul the average word length

In [ ]:
#avg_word_length = df.select(sum(col("word_lengths")).alias("total_length")).collect()[0]["total_length"] / df.count()

In [ ]:
#print(f"Average word length: {avg_word_length}")

In [ ]:
################################################################

In [ ]:
#spark = SparkSession.builder.appName("AverageWordLength").getOrCreate()

In [ ]:
#def avg_word(sentence):
#    words = sentence.split()
#    return sum(len(word) for word in words) / len(words)

In [ ]:
#avg_word_udf = udf(avg_word, DoubleType())

In [ ]:
#spark_df = spark_df.withColumn("average_word_length", avg_word_udf(col("Text")))

In [ ]:
#spark_df ["avg_word"] = spark_df ["Text"].apply(lambda x : avg_word(x))

In [ ]:
#avg_word_udf = udf(lambda sentence: avg_word(sentence), DoubleType())

In [ ]:
#Tweets = Tweets.withColumn("average_word_length", avg_word_udf(col("Text")))

In [ ]:
#Tweets.show()

In [ ]:
#****************************************************************************************************************
#*                                        S T R E M M I N G                                                     *
#****************************************************************************************************************

In [ ]:
st = PorterStemmer()

In [ ]:
# Define a Udf for Stremming

In [ ]:
def stem_text(text):
    stemmer = SnowballStemmer("english")
    words = nltk.word_tokenize(Text)
    stemmer_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

In [ ]:
# Register the UDF

In [ ]:
strem_udf = udf(stem_text, StringType())

In [ ]:
#Create a Spark session

In [ ]:
spark = SparkSession.builder.appName("StemmingExample").getOrCreate()


In [ ]:
#DataFrame 

In [ ]:
#spark_stemmed = spark_df.withColumn("Stemmed_Text", strem_udf(spark_df["Text"]))

In [ ]:
# Show te resulting DataFrame

In [ ]:
#spark_stemmed.show(truncate=False)

In [ ]:
#Data Procesising in panda

In [ ]:
Tweets_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [ ]:
Tweets_df = Tweets_df.withColumn("Text", lower(Tweets_df["text"]))


In [ ]:
Tweets_df = Tweets_df.withColumn("Text", regexp_replace(Tweets_df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
print(Tweets_df.columns)

In [ ]:
cols = ['Ids', 'date', 'User', 'Text']

In [ ]:
sns.set(style="darkgrid")

In [ ]:
sns.pairplot (Tweets_df[cols], heigt=2.5)

In [ ]:
df = df.withColumn("Text", lower(df["text"]))
df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
rows_df.shape

In [ ]:
# some excercises, it didn't work to me

In [ ]:
#header = tweets.first()

#tweets = tweets.filter(lambda row: row != header) 
#tweets = tweets.map(lambda row: [int(elem) for elem in row.split(',')])

In [ ]:
#tweets = sc.textFile("Tweets_path ")
#Header = tweets.first()

#tweets = tweets.filter(lambda row: row != header) 
#tweets = tweets.map(lambda row: [int(elem) for elem in row.split(',')])

In [ ]:
# creating the schema for my DataFrame
# was one  error because i didn' import pyspark.sql.types as typ
# but next did it, the command worked, great!

In [ ]:
#fields = [
#    *[
#        typ.StructField(h[1:-1], typ.IntegerType(), True)
#        for h in header.split(',')
#    ]
#]
#schema = typ.StructType(fields)

In [ ]:
# First put the Titles to see better

In [ ]:
#column_names  = ['_c0','Ids', 'Date', 'Flag', 'User', 'Text']    
#full_df = pd.read_csv('file:///home/hduser/Documents/CA2/ProjectTweets.csv', header=None, names=column_names)

In [ ]:
# rename the file whit  row's title  ******* ******* ****** ******* ****** ********

In [ ]:
#spark_df = "Title_pro_Tweets.csv"
#Check if the file already Exists
#if not os.path.exists("Title_pro_Tweets.csv"):
    #perform DataFrame operaction and save the file
#   spark_df.write.csv("file:///home/hduser/Documents/CA2/Title_pro_Tweets.csv", header=True)
#else:
#    print("file already exists. No need to save it again.")

In [ ]:
#schema = StructType().add("_c0","integer").add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [ ]:
#Schema for the  DataFrame
#schema = StructType([StructField("sentence", StringType(), True)]